In [3]:
import langchain
from langchain.schema import Document
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
import langchain_community

In [4]:
import os
import json
from datetime import datetime
import openai

In [5]:
def get_delivery_date(order_id: str) -> datetime:
    return datetime.today().strftime('%Y-%m-%d')

In [6]:
# LLM에서 사용할 함수 tool 생성
function_tool = {
    "type" : "function", # 도구의 타입을 function으로 설정 -> 함수 호출 기능 제공 명시
    "function" : {
        "name" : "get_delivery_date", # 함수 이름 정의
        "description" : "고객의 주문에 대한 배송 날짜를 확인합니다. 예를 들어, 고객이 '내 패키지가 어디에 있나요?'라고 물을 때 이 함수를 호출하세요.",
        "parameters" : {
            "type" : "object",
            "properties":{
                "order_id" : {
                "type" : "string",
                "description" : "고객의 주문 ID"
            }
         },
            "required" : ["order_id"],
            "additionalProperties" : False
        }

    }
}

tools = [function_tool]

In [7]:
messages = []
messages.append({"role": "system", "content": "당신은 도움이 되는 고객 지원 어시스턴트입니다. 제공된 도구를 사용하여 사용자를 지원하세요."})
messages.append({"role": "user", "content": "안녕하세요, 제 주문의 배송 날짜를 알려주실 수 있나요?"})
messages.append({"role": "assistant", "content": "안녕하세요! 제가 도와드릴 수 있습니다. 주문 ID를 알려주시겠어요?"})
messages.append({"role": "user", "content": "쓸데 없는 소리"})

In [ ]:
# 함수 매개변수 추출
tool_call = response.choices[0].tool_calls

if tool_call:
    print(tool_call[0])
else:
    print("No") #실제론 필요 없음

In [ ]:
arguments = json.loads(tool_call[0].function.arguments)
arguments

In [ ]:
get_delivery_date(**arguments)